In [ ]:
import pandas as pd

test_file = "common-sense/test_data.csv"
df_test = pd.read_csv(test_file)

In [ ]:
from transformers import AutoTokenizer, AutoModelForMultipleChoice
import torch

CHECKPOINT = "checkpoint-1000"
BERT = "bert-base-uncased"

bert_tokenizer = AutoTokenizer.from_pretrained(BERT)
bert_model = AutoModelForMultipleChoice.from_pretrained(f"./models/results/{BERT}/{CHECKPOINT}/")

In [ ]:
from transformers import AlbertTokenizer, AlbertForMultipleChoice

ALBERT = "albert-base-v2"
albert_tokenizer = AlbertTokenizer.from_pretrained(ALBERT)
albert_model = AlbertForMultipleChoice.from_pretrained(ALBERT)

In [ ]:
predictions = df_test.copy()

In [ ]:
options = ["OptionA", "OptionB", "OptionC"]
answers = ['A', 'B', 'C']

def predict(example, model, tokenizer):
    prompts = []
    for opt in options:
        prompts.append([example['FalseSent'], example[opt]])

    inputs = tokenizer(prompts, return_tensors="pt", padding=True)
    labels = torch.tensor(0).unsqueeze(0)
    
    outputs = model(**{k: v.unsqueeze(0) for k, v in inputs.items()}, labels=labels)

    return outputs

In [ ]:
predictions['bert'] = predictions.apply(lambda row: predict(row, bert_model, bert_tokenizer), axis=1)

In [ ]:
predictions['albert'] = predictions.apply(lambda row: predict(row, albert_model, albert_tokenizer), axis=1)

In [ ]:
def average(example, model_1, model_2):
    logits = (example[model_1].logits + example[model_2].logits) / 2

    predicted_class = logits.argmax().item()

    return answers[predicted_class]


predictions['label'] = predictions.apply(lambda row: average(row, "bert", "albert"), axis=1)

In [ ]:
predictions

In [ ]:
submission_df = predictions[["id", "label"]].copy()
submission_df = submission_df.rename(columns={"label": "answer"})
submission_df.set_index("id", inplace=True)
submission_df.to_csv(f"submissions/Submission_12-12-2022_1.csv")